In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

In [2]:
# Loading  data
df = pd.read_csv('data_balanced_cv.csv')

# Data Preparation

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Scaling Data

In [4]:
scaler = StandardScaler()

In [5]:
scaler.fit(df.drop('Y',axis=1))

StandardScaler()

In [6]:
scaled_features = scaler.fit_transform(df.drop('Y',axis=1))

Setting up datasets

In [7]:
X = pd.DataFrame(scaled_features,columns=df.columns[:-1]).values
y = df['Y'].values

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.regularizers import L1, L2, L1L2
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, cohen_kappa_score, roc_curve, classification_report, auc

In [14]:
def create_net(nodes_1 = 30, nodes_2 = 30, nodes_3 = 30):

    # create model
    dnn = Sequential()
        
    # hidden layer
    dnn.add(Dense(nodes_1, input_dim=X.shape[1], kernel_initializer='normal',kernel_regularizer=L2(0.01),activation='relu'))
    #dnn.add(Dropout(0.2))

    # hidden layer
    dnn.add(Dense(nodes_2, kernel_initializer='normal',kernel_regularizer=L2(0.01),activation='relu'))
    #dnn.add(Dropout(0.2))

    # hidden layer
    dnn.add(Dense(nodes_3, kernel_initializer='normal',kernel_regularizer=L2(0.01),activation='relu'))
    #dnn.add(Dropout(0.2))

    # output layer
    dnn.add(Dense(1, kernel_initializer='normal',kernel_regularizer=L2(0.01),activation='sigmoid'))

    # Compile model
    dnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return dnn

In [15]:
es = EarlyStopping(monitor='loss', patience=25, restore_best_weights=True)

In [19]:
# Wrap Keras model so it can be used by scikit-learn
dnn = KerasClassifier(
    build_fn=create_net,
    epochs=150, 
    batch_size=10,
    verbose=0)

In [21]:
# define search space
param_grid = dict()
param_grid['nodes_1'] = [20, 40, 60, 80, 100]
param_grid['nodes_2'] = [20, 40, 60, 80, 100]
param_grid['nodes_3'] = [20, 40, 60, 80, 100]

In [22]:
# define search
grid = GridSearchCV(dnn, param_grid, scoring='accuracy', n_jobs=-1, cv=3)

In [13]:
t0 = datetime.now()
grid_results = grid.fit(X, y, callbacks=es)
t1 = datetime.now()

Training Time:  0:05:14.530435


In [ ]:
# summarize results
print("Best: %f using %s" % (grid_results.best_score_, grid_results.best_params_))
print("Execution Time: ", (t1 - t0))